<a href="https://colab.research.google.com/github/juancanete86/ColabTensor2/blob/master/Colab_3_Crear_una_Red_Neuronal_Artificial_en_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Imagen cortesía de: https://www.kaggle.com/
</p>

## Paso 1: Instalar las dependencias y configurar el entorno de GPU

!pip install tensorflow-gpu==2.0.0.alpha0

## Paso 2: Importar las dependencias necesarias para el proyecto

In [1]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [2]:
tf.__version__

'2.1.0'

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [4]:
get_available_gpus()

['/device:GPU:0']

## Paso 3: Pre procesado de datos



### Cargar el dataset

In [5]:
#Cargar el dataset Fashion Mnist 
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### Normalizar las imágenes

Se divide cada imagen en los conjunto de entrenamiento y  de testing entre el valor máximo de cada uno de los píxeles (255).

De este modo, cada píxel se hallará en el rango [0, 1]. Al normalizar las imágenes, nos aseguramos que nuestro modelo de RNA entrenará más rápidamente.

In [6]:
X_train = X_train / 255.0

In [7]:
X_test = X_test / 255.0

### Redimensionar el dataset

Como vamos a utilizar una red neuronal totalmente conectada, vamos a redimensionar los subconjuntos de entrenamiento y testing a formato de vector en lugar de en formato de matriz.

In [8]:
#Como cada imagen tiene 28x28 píxeles, usamos la función reshape en todo el dataset de entrenamiento para convertirlo 
# en vectores de tamaño [-1 (todos los elementos), anchura * altura]
X_train = X_train.reshape(-1, 28*28)

In [9]:
X_train.shape

(60000, 784)

In [10]:
#Redimensionamos el conjunto de testing del mismo modo
X_test = X_test.reshape(-1, 28*28)

## Paso 4: Construir la Red Neuronal Artificial

### Definir el modelo

Simplemente se define un objeto de modelo Sequential.

In [103]:
model = tf.keras.models.Sequential()

### Añadir la primera capa totalmente conectada (capa Densa)

Hyper-parametros de la capa:
- número de unidades/neuronas: 128
- función de activación: ReLU
- input_shape: (784, )

In [104]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))

### Añadir una capa de Dropout 

Dropout es una técnica de Regularization donde aleatoriamente se asignan a ciertas neuronas de la red el valor cero. De este modo, mientras se entrena, estas neuronas no actualizarán sus valores. Al tener cierto porcentaje de neuronas sin actualizar, el proceso de entrenamiento toma más tiempo pero por contra tenemos menos posibilidades de sufrir overfitting.

In [105]:
model.add(tf.keras.layers.Dense(units=256, activation='tanh'))

In [106]:
model.add(tf.keras.layers.Dropout(0.25))

### Añadir la segunda capa (capa de salida)

- unidades: número de clases (10 en el caso del Fashion MNIST)
- función de activación: 'softmax' (probabilidades de cada clase)

In [107]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy 

In [108]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, name='Adam')

In [109]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [110]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_26 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                2570      
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

In [111]:
model.fit(X_train, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4939 - sparse_categorical_accuracy: 0.8208
Epoch 2/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3768 - sparse_categorical_accuracy: 0.8623
Epoch 3/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3386 - sparse_categorical_accuracy: 0.8755
Epoch 4/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3167 - sparse_categorical_accuracy: 0.8831
Epoch 5/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3024 - sparse_categorical_accuracy: 0.8868
Epoch 6/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2877 - sparse_categorical_accuracy: 0.8929
Epoch 7/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2771 - sparse_categorical_accuracy: 0.8966
Epoch 8/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.2665 - spars

### Evaluación del modelo y predicción

In [102]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 43us/sample - loss: 0.3317 - sparse_categorical_accuracy: 0.8832


In [112]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 33us/sample - loss: 0.3370 - sparse_categorical_accuracy: 0.8837


In [113]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.8837000131607056


## Paso 5 : Guardar el modelo

### Guardar la arquitectura (topoligía) de la red neuronal

In [114]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Guardar los pesos de la red neuronal

In [115]:
model.save_weights("fashion_model.h5")